# 2.2 WaveFunction Overlap - Randomized Measurement

The `WaveFunctionOverlap` or `EchoListen` instance provides three methods:

1. **`EchoListenRandomized`**: Uses randomized measurements and serves as the default method.
   ```python
   exp_method_02 = WaveFunctionOverlap()
   ```
   or
   ```python
   exp_method_02 = EchoListen()
   ```
   or
   ```python
   exp_method_02 = WaveFunctionOverlap(method="randomized")
   ```
   or
   ```python
   exp_method_02 = EchoListen(method="randomized")
   ```
   or
   ```python
   exp_method_02 = WaveFunctionOverlap(method="base")
   ```
   or
   ```python
   exp_method_02 = EchoListen(method="base")
   ```
2. **`EchoListenHadamard`**: Employs the Hadamard test for entropy calculation.
   ```python
   exp_method_01 = WaveFunctionOverlap(method="hadamard")
   ```
   or
   ```python
   exp_method_01 = EchoListen(method="hadamard")
   ```
3. **`EchoListenRandomizedV1`**: Implements randomized measurements with a different approach but is deprecated due to limitations in subsystem selection.
   ```python
   exp_method_03 = WaveFunctionOverlap(method="randomized_v1")
   ```
   or
   ```python
   exp_method_03 = EchoListen(method="randomized_v1")
   ```

All three methods calculate the entropy of a quantum state, but their calculation techniques and post-processing procedures are entirely different.


## 2.2 Randomized Measurement

This method is based on [Probing Rényi entanglement entropy via randomized measurements](https://www.science.org/doi/abs/10.1126/science.aau4963) with deplorizing error mitigation by [Simple mitigation of global depolarizing errors in quantum simulations](https://link.aps.org/doi/10.1103/PhysRevE.104.035309).


---

### `randomized_overlap_echo`

<a id='randomized_overlap_echo'></a>


Here, we assume you already have the calculated data. You can call the function `randomized_overlap_echo` from `qurry.process.randomized_measure` to calculate the entropy.

This function requires only three arguments: `shots`, `counts`, and `selected_classical_registers`. All other arguments are optional.

Note that `selected_classical_registers` expects a list of <u>bitstring indices</u>, **NOT** <u>qubit indices</u>, as these are distinct.

The following is the arguments of the function:

```python

def randomized_overlap_echo(
    shots: int,
    first_counts: list[dict[str, int]],
    second_counts: list[dict[str, int]],
    selected_classical_registers: Optional[Iterable[int]] = None,
    backend: PostProcessingBackendLabel = DEFAULT_PROCESS_BACKEND,
    pbar: Optional[tqdm.tqdm] = None,
) -> WaveFuctionOverlapResult:
    """Calculate wavefunction overlap
    a.k.a. loschmidt echo when processes time evolution system.

    Args:
        shots (int):
            Shots of the experiment on quantum machine.
        first_counts (list[dict[str, int]]):
            Counts of the experiment on quantum machine.
        second_counts (list[dict[str, int]]):
            Counts of the experiment on quantum machine.
        selected_classical_registers (Optional[Iterable[int]], optional):
            The list of **the index of the selected_classical_registers**.
        backend (ExistingProcessBackendLabel, optional):
            Backend for the process. Defaults to DEFAULT_PROCESS_BACKEND.
        pbar (Optional[tqdm.tqdm], optional):
            The progress bar API, you can use put a :cls:`tqdm` object here.
            This function will update the progress bar description.
            Defaults to None.


    Returns:
        WaveFuctionOverlapResult: A dictionary contains purity, entropy,
            a list of each overlap, puritySD, degree, actual measure range, bitstring range.
    """
```
This function returns a dictionary that contains the echo, echoSD, and other relevant information. 
The returned `dict` contains the following information:

```python
class WaveFuctionOverlapResult(TypedDict):
    """The return type of the post-processing for wavefunction overlap."""

    echo: np.float64
    """The overlap value."""
    echoSD: np.float64
    """The overlap standard deviation."""
    echoCells: dict[int, np.float64]
    """The overlap of each single count."""
    num_classical_registers: int
    """The number of classical registers."""
    classical_registers: Optional[list[int]]
    """The list of the index of the selected classical registers."""
    classical_registers_actually: list[int]
    """The list of the index of the selected classical registers which is actually used."""
    # refactored
    counts_num: int
    """The number of first counts and second counts."""
    taking_time: float
    """The calculation time."""
```




---

#### Dummy Data


In [1]:
from qurry.capsule import quickRead

easy_dummy: dict[str, dict[str, int]] = quickRead("../easy-dummy.json")
large_dummy_list = [easy_dummy["0"] for _ in range(100)]

---

#### Simple Example


In [2]:
from qurry.process.randomized_measure import (
    randomized_overlap_echo,
)

test_result_1_1_1 = randomized_overlap_echo(
    4096, large_dummy_list, large_dummy_list, list(range(6))
)

In [3]:
from pprint import pprint

print("| result of randomized_overlap_echo except for purityCells")
pprint({k: v for k, v in test_result_1_1_1.items() if k != "echoCells"})
# "echoCells" is too long we skip it here

print("| result of randomized_overlap_echo[purityCells]")
print(test_result_1_1_1["echoCells"][0])
print(test_result_1_1_1["echoCells"][1])

| result of randomized_overlap_echo except for purityCells
{'classical_registers': [0, 1, 2, 3, 4, 5],
 'classical_registers_actually': [0, 1, 2, 3, 4, 5],
 'counts_num': 100,
 'echo': np.float64(1.0627930164337158),
 'echoSD': np.float64(0.0),
 'num_classical_registers': 8,
 'taking_time': 0.007889697}
| result of randomized_overlap_echo[purityCells]
1.0627930164337158
1.0627930164337158


---

#### Integration with your own progress bar


In [4]:
from tqdm import tqdm
import time

all_counts_progress_01 = tqdm(
    [
        (4096, large_dummy_list, list(range(6))),
        (4096, large_dummy_list, list(range(2, 8))),
        (4096, large_dummy_list, list(range(7))),
        (4096, large_dummy_list, list(range(0, 7))),
        (4096, large_dummy_list, [0, 1, 2, 6, 7]),
        (4096, large_dummy_list, [3, 4, 5, 6, 7, 0, 1]),
    ],
    bar_format="| {desc} - {elapsed} < {remaining}",
)

test_result_1_1_2 = []
for tmp_shot, tmp_counts, tmp_partition in all_counts_progress_01:
    print(tmp_partition)
    test_result_1_1_2.append(
        randomized_overlap_echo(
            tmp_shot, tmp_counts, tmp_counts, tmp_partition, pbar=all_counts_progress_01
        )
    )
    time.sleep(0.5)

| Calculate selected classical registers: [0, 1, 2, 3, 4, 5]. - 00:00 < ?

[0, 1, 2, 3, 4, 5]


| Calculate selected classical registers: [2, 3, 4, 5, 6, 7]. - 00:00 < 00:02

[2, 3, 4, 5, 6, 7]


| Calculate selected classical registers: [0, 1, 2, 3, 4, 5, 6]. - 00:01 < 00:02

...

| Calculate selected classical registers: [0, 1, 2, 6, 7]. - 00:02 < 00:01      

[0, 1, 2, 6, 7]


| Calculate selected classical registers: [3, 4, 5, 6, 7, 0, 1]. - 00:02 < 00:00

[3, 4, 5, 6, 7, 0, 1]


| Calculate selected classical registers: [3, 4, 5, 6, 7, 0, 1]. - 00:03 < 00:00


---

#### Using Python backend

It will be slow. You'd better think twice before using it.


In [5]:
all_counts_progress_02 = tqdm(
    [
        (4096, large_dummy_list, list(range(6))),
        (4096, large_dummy_list, list(range(2, 8))),
        (4096, large_dummy_list, list(range(7))),
        (4096, large_dummy_list, list(range(0, 7))),
        (4096, large_dummy_list, [0, 1, 2, 6, 7]),
        (4096, large_dummy_list, [3, 4, 5, 6, 7, 0, 1]),
    ],
    bar_format="| {desc} - {elapsed} < {remaining}",
)


test_result_1_1_2 = []
for tmp_shot, tmp_counts, tmp_partition in all_counts_progress_02:
    print(tmp_partition)
    test_result_1_1_2.append(
        randomized_overlap_echo(
            tmp_shot,
            tmp_counts,
            tmp_counts,
            tmp_partition,
            pbar=all_counts_progress_02,
            backend="Python",
        )
    )
    time.sleep(0.5)

| Calculate selected classical registers: [0, 1, 2, 3, 4, 5]. - 00:00 < ?

[0, 1, 2, 3, 4, 5]


| Calculate selected classical registers: [2, 3, 4, 5, 6, 7]. - 00:01 < 00:06

[2, 3, 4, 5, 6, 7]


| Calculate selected classical registers: [0, 1, 2, 3, 4, 5, 6]. - 00:02 < 00:05

...

| Calculate selected classical registers: [0, 1, 2, 6, 7]. - 00:08 < 00:04      

[0, 1, 2, 6, 7]


| Calculate selected classical registers: [3, 4, 5, 6, 7, 0, 1]. - 00:09 < 00:01

[3, 4, 5, 6, 7, 0, 1]


| Calculate selected classical registers: [3, 4, 5, 6, 7, 0, 1]. - 00:11 < 00:00


---

### Post-Processing Speed-up Availablities and Version Info



In [6]:
from qurry.process.status import AVAIBILITY_STATESHEET

AVAIBILITY_STATESHEET

 | Qurry version: 0.10.0
--------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability ................... Python Cython Rust  
 - randomized_measure
   - entangled_entropy.entropy_core_2 ....... Yes    Depr.  Yes   
   - entangle_entropy.purity_cell_2 ......... Yes    Depr.  Yes   
   - wavefunction_overlap.echo_core_2 ....... Yes    Depr.  Yes   
   - wavefunction_overlap.echo_cell_2 ....... Yes    Depr.  Error 
 - utils
   - randomized ............................. Yes    Depr.  Yes   
   - construct .............................. Yes    No     Yes   
   - dummy .................................. Yes    No     Yes   
   - test ................................... Yes    No     Yes   
 - hadamard_test
   - purity_echo_core ....................... Yes    No     Yes   
 - magnet_square
   - magnsq_core ............................ Yes    No     No    
--------------------------------------------------------
   + Yes ...... Working normally